In [1]:
import random, copy, time
import numpy as np

In [2]:
# crossover
def crossover(parent1, parent2):
    pos = random.randint(0, len(parent1)-1) #randomly cut point
    child1 = parent1[0:pos] + parent2[pos:]
    child2 = parent2[0:pos] + parent1[pos:]
    return [child1, child2]
 
# mutation
def mutation(parent):
    i, j = random.sample(range(0, len(parent)-1), 2) #randomly mutation
    parent[i], parent[j] = parent[j], parent[i]
    child = parent
    return [child]

#count fitness
def count_fitness(individu):
    hargabeliperkg = [3200, 5500, 1500, 3000, 8000, 9000, 5000, 2500]
    angkutan = 2000000
    semenkapur = 100000
    tenagakerjaangkut = 500000
    wadah = 200000
    tenagakerjakemas = 150000
    sewalapak = 20000
    listrik = 3000
    sampah = 2000
    danatakterduga = 100000

    biayapembelian = np.multiply(individu, hargabeliperkg)
    totalbiayabeli = sum(biayapembelian)

    biayakapasitaslebih = (sum(individu)-5000)*100 if (sum(individu)-5000)*100 > 0 else 0 

    TC = sum([totalbiayabeli, angkutan, semenkapur, tenagakerjaangkut, 
              wadah, tenagakerjakemas, sewalapak, listrik, sampah, danatakterduga,
              biayakapasitaslebih])

    permintaan = [3000, 1000, 1500, 160, 180, 100, 120, 100]
    hargajualperkg = [3700, 6500, 2000, 5000, 9000, 11000, 6000, 4000]
    biayapenjualan = np.multiply(permintaan, hargajualperkg)

    TR = sum(biayapenjualan)

    stok = [a-b for a,b in zip(individu,permintaan)]

    hargajualperstok = [3500, 6000, 1000, 4500, 8500, 10000, 5000, 3000]
    biayajualstok = np.multiply(stok, hargajualperstok)

    TRstok = sum(biayajualstok)
    TRcurrent = TR + TRstok

    biayajualmax = np.multiply(individu,hargajualperkg)
    TRmax = sum(biayajualmax)
    TPF = TRmax - TC
    potensirugi = TRcurrent - TC

    fit_value = 0 if (TPF-potensirugi)==0 else 1000/(TPF-potensirugi)
    return fit_value
    
def set_fitness(individu): 
    fitness = []
    for i in range(0, len(individu)):
        fitness.append(count_fitness(individu[i]))
    return fitness

#selection
def selection(individu, popsize):
    fitness = set_fitness(individu)
    for i in range(0, len(individu)):
        individu[i].insert(0, fitness[i])
    individu = sorted(individu)
    for i in range(len(individu)):
        individu[i].pop(0)
    individu = individu[-popsize:]
    return individu

#get best solution
def get_best(individu):
    f = set_fitness(individu)
    best = max(f)
    index = f.index(best)
    return individu[index]

#mendapatkan solusi terbaik dari sub populasi GA
def get_the_best(individu):
    f = set_fitness(individu)
    best = max(f)
    index = f.index(best)
    print("Solusi terbaik =", individu[index], " fitness =", best)
        
# main GA
def GeneticAlgorithm(data, chromosome, popsize, cr, mr, generation, subpop):
    individu = []
    fitness = []
    best_individu = []
    epoch = 0
    sp = []
    bsp = []
    bf = []
    
    #init individu in popsize in subpop
    for k in range (subpop):
        for i in range(popsize):
            individu.append(random.choices(data, k=chromosome))
        sp.append(individu)
        individu = []
    
    while (epoch < generation):
        best_sp = []
        for i in range (subpop):
            individu = copy.deepcopy(sp[i])
            #crossover
            do_crossover = cr*popsize
            for i in range(0, round(do_crossover/2)):
                individu.extend(crossover(individu[random.randint(0, popsize-1)], individu[random.randint(0, popsize-1)]))
            if (round(do_crossover/2)>do_crossover/2):
                individu.pop(-1)

            #mutation
            do_mutation = mr*popsize
            for i in range(0, round(do_mutation)):
                individu.extend(mutation(individu[random.randint(0, popsize-1)]))
        
            #selection
            individu = selection(individu, popsize)
            sp[i] = copy.deepcopy(individu)
            best_sp.append(get_best(individu))
            bsp = copy.deepcopy(best_sp)
            
        #migrasi
        for i in range(subpop):
            for j in range(popsize):
                if i is subpop-1:
                    if sp[i][j] == bsp[i]:
                        sp[i][j] = copy.deepcopy(bsp[0])
                else :
                    if sp[i][j] == bsp[i]:
                        sp[i][j] = copy.deepcopy(bsp[i+1])
                
        epoch = epoch + 1
    
    #mendapatkan terbaik dari semua subpop
    for i in range(subpop):
        bf.append(get_best(sp[i]))
    get_the_best(bf)

In [ ]:
data = [i * 10 for i in range(501)] #random number between 0 - 5000 by interval 10

#parameter
populationsize = 10
gen_length = 8
cr = 0.4
mr = 0.6
generation = 10000
sub_pop = 10

%time GeneticAlgorithm(data, gen_length, populationsize, cr, mr, generation, sub_pop)